In [1]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str = "Jane Doe"
    age: int

person = Person(name="John", age="3")
print(person.name)
print(person.age, type(person.age))


user = Person(age=30)

print(user)

print(user.model_dump())

John
3 <class 'int'>
name='Jane Doe' age=30
{'name': 'Jane Doe', 'age': 30}


In [2]:
from typing import Optional, List

class User(BaseModel):
    name: str
    age: Optional[int] = None
    friends: List[str] = []

user = User(name="John", friends=["Jane", "Doe"])

print(user)
print(user.model_dump())

name='John' age=None friends=['Jane', 'Doe']
{'name': 'John', 'age': None, 'friends': ['Jane', 'Doe']}


In [3]:
# more complex example

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip: str

class User(BaseModel):
    name: str
    age: Optional[int] = None
    address: List[Address] = []

user = User(name="John", age=30, address=[Address(street="123 Main St", city="Anytown", state="CA", zip="12345")])

print(user)
print(user.model_dump())

name='John' age=30 address=[Address(street='123 Main St', city='Anytown', state='CA', zip='12345')]
{'name': 'John', 'age': 30, 'address': [{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}]}


In [4]:
# Define model
from typing import List, Dict, Optional, Union

class Item(BaseModel):
    name: str
    price: float
    tags: List[str]
    metadata: Dict[str, Union[str, int, float]]

item = Item(name="Laptop", price=1000, tags=["electronics", "laptop"], metadata={"brand": "Dell", "model": "Inspiron 15", 'version': 1.0})

print(item)
print(item.model_dump())


name='Laptop' price=1000.0 tags=['electronics', 'laptop'] metadata={'brand': 'Dell', 'model': 'Inspiron 15', 'version': 1.0}
{'name': 'Laptop', 'price': 1000.0, 'tags': ['electronics', 'laptop'], 'metadata': {'brand': 'Dell', 'model': 'Inspiron 15', 'version': 1.0}}


In [5]:
from pydantic import BaseModel, conint, constr

class User(BaseModel):
    name: constr(min_length=3, max_length=50)
    age: conint(gt=0, lt=120)

user = User(name="John", age=110)

# user = User(name="John", age=133)
print(user)
print(user.model_dump())

name='John' age=110
{'name': 'John', 'age': 110}


In [6]:
# create validator

from pydantic import field_validator

class User(BaseModel):
    name: constr(min_length=3, max_length=50)
    age: int

    @field_validator('age')
    def check_age(cls, v):
        if v < 0:
            raise ValueError("Age must be greater than 0")
        return v


# user = User(name="John", age=-1)
# print(user)
# print(user.model_dump())

In [7]:
from pydantic import BaseModel, field_validator


class Person(BaseModel):
    first_name: str
    last_name: Optional[str] = None

    @field_validator('first_name', 'last_name')
    def names_cannot_be_empty(cls, value):
        print(value)
        if value is None:
            raise ValueError('Name fields cannot be empty')
        return value

person = Person(first_name="John")

John


In [8]:
from pydantic import Field, BaseModel
from datetime import date

class DateRange(BaseModel):
    chain_of_thought: str = Field(
        description="Reasoning behind the date range.", exclude=True)
    start: date
    end: date

date_range = DateRange(
    chain_of_thought="""
    I want to find the date range for the last 30 days.
    Today is 2025-01-07. Therefore the start date is 2024-12-08.
    """,
    start=date(2024, 12, 8),
    end=date(2025, 1, 7)
)

print(date_range)
print(date_range.model_dump())


chain_of_thought='\n    I want to find the date range for the last 30 days.\n    Today is 2025-01-07. Therefore the start date is 2024-12-08.\n    ' start=datetime.date(2024, 12, 8) end=datetime.date(2025, 1, 7)
{'start': datetime.date(2024, 12, 8), 'end': datetime.date(2025, 1, 7)}


In [9]:
date_range.model_json_schema()

{'properties': {'chain_of_thought': {'description': 'Reasoning behind the date range.',
   'title': 'Chain Of Thought',
   'type': 'string'},
  'start': {'format': 'date', 'title': 'Start', 'type': 'string'},
  'end': {'format': 'date', 'title': 'End', 'type': 'string'}},
 'required': ['chain_of_thought', 'start', 'end'],
 'title': 'DateRange',
 'type': 'object'}

In [10]:
from pydantic import BaseModel, Field
from pydantic.json_schema import SkipJsonSchema
from typing import Union

class Response(BaseModel):
    question: str = Field(description="The question asked by the user.")
    private_field: SkipJsonSchema[Union[str, int]] = None

response = Response(question="What is the capital of France?")
print(response)
print(response.model_dump())

response.model_json_schema()


question='What is the capital of France?' private_field=None
{'question': 'What is the capital of France?', 'private_field': None}


{'properties': {'question': {'description': 'The question asked by the user.',
   'title': 'Question',
   'type': 'string'}},
 'required': ['question'],
 'title': 'Response',
 'type': 'object'}

In [11]:
from pydantic import BaseModel, Field, SecretStr

class User(BaseModel):
    age: int = Field(description="The age of the user.")
    name: str = Field(title="Username")
    password: str = Field(
        json_schema_extra={
            "title": "Password",
            "description": "The password of the user.",
            "examples": ["password123", "secret123"]
        }
    )
user = User(age=30, name="john_doe", password="hehe zz")

user.model_json_schema()


{'properties': {'age': {'description': 'The age of the user.',
   'title': 'Age',
   'type': 'integer'},
  'name': {'title': 'Username', 'type': 'string'},
  'password': {'description': 'The password of the user.',
   'examples': ['password123', 'secret123'],
   'title': 'Password',
   'type': 'string'}},
 'required': ['age', 'name', 'password'],
 'title': 'User',
 'type': 'object'}

In [12]:
cd ..

/home/vule/projects/learn_agent


In [32]:
from src.key import openai_api_key

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini", temperature=1)


In [41]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

class Joke(BaseModel):
    setup: str = Field(description="question to setup of the joke.")
    punchline: str = Field(description="answer to punchline of the joke.", min_length=4)

joke_parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": joke_parser.get_format_instructions()}
)

generator = prompt | llm | joke_parser

message = generator.invoke({"query": "What is the setup of the joke?"})

print(message)

setup='Why did the chicken cross the road?' punchline='To get to the other side!'


In [42]:
message

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [45]:
# Example create a prompt to get the actors and their films

# Import field
from pydantic import Field


class Film(BaseModel):
    title: str = Field(description="The title of the film.")
    year: int = Field(description="The year of the film.")
    director: str = Field(description="The director of the film.")

class Actor(BaseModel):
    name: str = Field(description="The name of the actor.")
    films: List[Film] = Field(description="The films of the actor.")

actor_parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": actor_parser.get_format_instructions()}
)

generator = prompt | llm | actor_parser

message = generator.invoke({"query": "What are the films of Tom Hanks?"})

print(message)

name='Tom Hanks' films=[Film(title='Forrest Gump', year=1994, director='Robert Zemeckis'), Film(title='Saving Private Ryan', year=1998, director='Steven Spielberg'), Film(title='Cast Away', year=2000, director='Robert Zemeckis'), Film(title='Toy Story', year=1995, director='John Lasseter'), Film(title='The Green Mile', year=1999, director='Frank Darabont')]


In [48]:
for o in message.films:
    print(f"{o.title} ({o.year}) directed by {o.director}")

Forrest Gump (1994) directed by Robert Zemeckis
Saving Private Ryan (1998) directed by Steven Spielberg
Cast Away (2000) directed by Robert Zemeckis
Toy Story (1995) directed by John Lasseter
The Green Mile (1999) directed by Frank Darabont
